<a href="https://colab.research.google.com/github/amitxsethi/Text2Sql-AIML/blob/main/gensql_group6_amit_llama7B_full_spider.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

import torch
print(f"\nCUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

Tue Nov 18 16:02:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

The `nbstripout` tool can help clean up notebook metadata, which might resolve the rendering issue. You'll need to specify the exact filename of this notebook. After running the command, you might need to refresh your browser page for the changes to take effect.

In [2]:
!git clone https://github.com/jieshi10/gensql.git
%cd gensql
!ls -la
print("✓ Repository cloned!")

Cloning into 'gensql'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 149 (delta 29), reused 148 (delta 28), pack-reused 0 (from 0)
Receiving objects: 100% (149/149), 2.42 MiB | 13.34 MiB/s, done.
Resolving deltas: 100% (29/29), done.
/content/gensql
total 132
drwxr-xr-x 10 root root  4096 Nov 18 16:02 .
drwxr-xr-x  1 root root  4096 Nov 18 16:02 ..
drwxr-xr-x  4 root root  4096 Nov 18 16:02 benchmark
drwxr-xr-x  3 root root  4096 Nov 18 16:02 bird_code
drwxr-xr-x  2 root root  4096 Nov 18 16:02 docker
drwxr-xr-x  8 root root  4096 Nov 18 16:02 .git
-rw-r--r--  1 root root  3139 Nov 18 16:02 .gitignore
-rw-r--r--  1 root root  1064 Nov 18 16:02 LICENSE
-rw-r--r--  1 root root 37953 Nov 18 16:02 main.py
drwxr-xr-x  2 root root  4096 Nov 18 16:02 pretrained
-rw-r--r--  1 root root 16684 Nov 18 16:02 process_sql.py
-rw-r--r--  1 root root  4259 Nov 18 16:02 README.md
-rw-r--r--  1 r

In [3]:
# Fix dependency issues first
!pip install -q jedi  # Fix ipython warning

# Install packages in the right order to avoid conflicts
print("Installing packages (this takes 2-3 minutes)...")

# Step 1: Core ML packages
!pip install -q torch torchvision torchaudio

# Step 2: NLP packages with compatible versions
!pip install -q 'transformers>=4.41.0,<5.0.0'
!pip install -q sentence-transformers
!pip install -q 'nltk>=3.9'  # Fix textblob conflict
!pip install -q openai

# Step 3: SQL and utility packages
!pip install -q 'sqlglot>=23.4'  # Use newer version
!pip install -q sqlparse
!pip install -q func-timeout
!pip install -q scikit-learn

# Step 4: Install vLLM last (takes longest)
print("\nInstalling vLLM (this takes 1-2 minutes)...")
!pip install -q vllm

print("\n✓ All packages installed successfully!")
print("\nInstalled versions:")
!pip list | grep -E 'torch|transformers|vllm|nltk|sqlglot'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.4 MB/s eta 0:00:00
Installing packages (this takes 2-3 minutes)...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done

Installing vLLM (this takes 1-2 minutes)...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.2/438.2 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 126.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 125.1 MB/s eta 0:00

In [4]:
import nltk
import ssl

ssl._create_default_https_context = ssl._create_unverified_context
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

print("✓ NLTK data downloaded!")

✓ NLTK data downloaded!


In [5]:
print("Fixing sentence_transformers compatibility...")

# Fix SentenceTransformer.py
with open('sentence_transformers/SentenceTransformer.py', 'r') as f:
    content = f.read()
content = content.replace(
    'from huggingface_hub import HfApi, HfFolder, Repository, hf_hub_url, cached_download',
    'from huggingface_hub import HfApi, HfFolder, Repository, hf_hub_url, hf_hub_download'
)
with open('sentence_transformers/SentenceTransformer.py', 'w') as f:
    f.write(content)

# Fix util.py
with open('sentence_transformers/util.py', 'r') as f:
    content = f.read()

content = content.replace(
    'from huggingface_hub import HfApi, hf_hub_url, cached_download, HfFolder',
    '''from huggingface_hub import HfApi, hf_hub_url, hf_hub_download, HfFolder
try:
    from huggingface_hub import cached_download
except ImportError:
    cached_download = None'''
)

with open('sentence_transformers/util.py', 'w') as f:
    f.write(content)

print("✓ Fixed sentence_transformers!")

Fixing sentence_transformers compatibility...
✓ Fixed sentence_transformers!


In [6]:
from huggingface_hub import snapshot_download

print("Downloading BGE model (~1.3 GB)...")
print("This will take 2-3 minutes...\n")

model_path = snapshot_download(
    repo_id='BAAI/bge-large-en-v1.5',
    local_dir='./pretrained/bge-large-en-v1.5',
    local_dir_use_symlinks=False
)

print(f"\n✓ BGE model downloaded!")
!du -sh pretrained/bge-large-en-v1.5

This will take 2-3 minutes...



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

onnx/model.onnx:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

...

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]


✓ BGE model downloaded!
3.8G	pretrained/bge-large-en-v1.5


In [7]:
import os

os.makedirs('benchmark', exist_ok=True)

print("Downloading Spider dataset (~100 MB)...")
!wget -q --show-progress -O benchmark/spider.zip "https://drive.usercontent.google.com/download?id=1TqleXec_OykOYFREKKtschzY29dUcVAQ&export=download&confirm=t"

print("\nExtracting...")
!cd benchmark && unzip -q spider.zip && rm spider.zip

print("✓ Spider dataset ready!")
!du -sh benchmark/spider
!ls benchmark/spider/

benchmark/spider.zi 100%[===================>]  95.12M   134MB/s    in 0.7s    

Extracting...
✓ Spider dataset ready!
878M	benchmark/spider
database      dev.json	  tables.json	  train_others.json
dev_gold.sql  README.txt  train_gold.sql  train_spider.json


In [8]:
from huggingface_hub import snapshot_download
import os

print("="*70)
print("DOWNLOADING CODELLAMA-7B-INSTRUCT")
print("="*70)
print("\nModel size: ~13 GB")
print("Time: 10-15 minutes\n")

model_path = snapshot_download(
    repo_id='codellama/CodeLlama-7b-Instruct-hf',
    local_dir='./models/CodeLlama-7b-Instruct',
    local_dir_use_symlinks=False
)

print(f"\n✓ CodeLlama-7B downloaded!")
print(f"Location: {model_path}")

# Check size
!du -sh ./models/CodeLlama-7b-Instruct

DOWNLOADING CODELLAMA-7B-INSTRUCT

Model size: ~13 GB
Time: 10-15 minutes



Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

LICENSE: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]


✓ CodeLlama-7B downloaded!
Location: /content/gensql/models/CodeLlama-7b-Instruct
26G	./models/CodeLlama-7b-Instruct


In [9]:
!lsof -ti :8000 && echo "⚠️  Port in use" || echo "✓ Port free"

✓ Port free


In [10]:
# Update the LLM client to use port 8000
with open('main.py', 'r') as f:
    code = f.read()

# Change port from 8080 to 8000
code = code.replace("port=8080", "port=8000")
code = code.replace("LLMClient(host='127.0.0.1', port=8080)",
                   "LLMClient(host='127.0.0.1', port=8000)")
code = code.replace("DB_NAME: str='bird'", "DB_NAME: str='spider'")

with open('main.py', 'w') as f:
    f.write(code)

print("✓ Updated main.py to use port 8000")

✓ Updated main.py to use port 8000


In [11]:
import subprocess
import time
import requests

# Kill any old processes
print("Stopping old vLLM processes...")
subprocess.run("pkill -9 -f vllm", shell=True)
time.sleep(3)
print("✓ Stopped\n")

print("="*70)
print("STARTING CODELLAMA-7B ON A100 (40GB)")
print("="*70)
print("\n⚙️ Settings:")
print("   • Model: ./models/CodeLlama-7b-Instruct")
print("   • Max context: 2048 tokens")
print("   • GPU memory: 0.85")
print("   • Max sequences: 16")
print("\n🕐 Loading model (60-90 seconds)...\n")

# Start vLLM with correct path
subprocess.Popen([
    'python', '-m', 'vllm.entrypoints.openai.api_server',
    '--host', '127.0.0.1',
    '--port', '8000',
    '--model', './models/CodeLlama-7b-Instruct',  # ← Correct path
    '--served-model-name', 'CodeLlama-7B-Instruct',
    '--max-model-len', '2048',
    '--gpu-memory-utilization', '0.85',
    '--dtype', 'float16',
    '--max-num-seqs', '16',
    '--disable-log-requests'
], stdout=open('vllm_codellama.log', 'w'), stderr=subprocess.STDOUT)

# Wait for server
print("Waiting for server to start...")
for i in range(15):
    time.sleep(10)
    try:
        r = requests.get('http://127.0.0.1:8000/health', timeout=2)
        if r.status_code == 200:
            models = requests.get('http://127.0.0.1:8000/v1/models').json()
            print(f"\n{'='*70}")
            print(f"✓ SERVER READY after {(i+1)*10} seconds!")
            print(f"✓ Model: {models['data'][0]['id']}")
            print('='*70)
            print("\n🎉 Ready to run Gen-SQL!\n")
            break
    except:
        print(f"⏳ {(i+1)*10}s...")
else:
    print("\n❌ Timeout. Checking logs:\n")
    subprocess.run("tail -30 vllm_codellama.log", shell=True)

Stopping old vLLM processes...
✓ Stopped

STARTING CODELLAMA-7B ON A100 (40GB)

⚙️ Settings:
   • Model: ./models/CodeLlama-7b-Instruct
   • Max context: 2048 tokens
   • GPU memory: 0.85
   • Max sequences: 16

🕐 Loading model (60-90 seconds)...

Waiting for server to start...
⏳ 10s...
⏳ 20s...
⏳ 30s...
⏳ 40s...
⏳ 50s...
⏳ 60s...
⏳ 70s...
⏳ 80s...
⏳ 90s...
⏳ 100s...

✓ SERVER READY after 110 seconds!
✓ Model: CodeLlama-7B-Instruct

🎉 Ready to run Gen-SQL!



In [ ]:
!tail -100 vllm_codellama.log

2025-11-18 09:59:30.468084: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-18 09:59:30.485134: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763459970.506294    6814 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763459970.512725    6814 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763459970.529095    6814 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

✓ Updated main.py to use port 8000


In [ ]:
!tail -50 vllm_codellama.log

(EngineCore_DP0 pid=7073) INFO 11-18 10:01:03 [kv_cache_utils.py:1091] Maximum concurrency for 2,048 tokens per request: 20.79x
Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 7/7 [00:00<00:00, 17.00it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 5/5 [00:00<00:00, 20.83it/s]
(EngineCore_DP0 pid=7073) INFO 11-18 10:01:04 [gpu_model_runner.py:3480] Graph capturing finished in 2 secs, took 0.11 GiB
(EngineCore_DP0 pid=7073) INFO 11-18 10:01:04 [core.py:210] init engine (profile, create kv cache, warmup model) took 39.52 seconds
(APIServer pid=6814) INFO 11-18 10:01:05 [loggers.py:147] Engine 000: vllm cache_config_info with initialization after num_gpu_blocks is: 2661
(APIServer pid=6814) INFO 11-18 10:01:05 [api_server.py:1634] Supported_tasks: ['generate']
(APIServer pid=6814) INFO 11-18 10:01:05 [api_server.py:1912] Starting vLLM API server 0 on http://127.0.0.1:8000
(APIServer pid=6814) INFO 11-18 10:01:05 [launcher.py:34] Available routes are:
(

In [ ]:
!grep -n "spider"  main.py | head -20

79:DB_NAME: str='spider'
80:# DB_NAME: str='spider'
94:elif DB_NAME == 'spider':
95:    # spider
96:    DB_SET_PATH: str='benchmark/spider/database/*'
97:    # DB_SET_PATH: str='spider_code/spider_ext/*'
98:    TRAINING_SET_PATHS: List[str]=['benchmark/spider/train_others.json', 'benchmark/spider/train_spider.json']
100:    DEV_SET_PATH: str='benchmark/spider/dev.json'


In [12]:
import nltk
import ssl

# Fix SSL
ssl._create_default_https_context = ssl._create_unverified_context

# Download the new punkt_tab (required for NLTK 3.9+)
print("Downloading punkt_tab...")
nltk.download('punkt_tab', quiet=False)
nltk.download('averaged_perceptron_tagger_eng', quiet=False)

# Also download the old ones as backup
nltk.download('punkt', quiet=False)
nltk.download('averaged_perceptron_tagger', quiet=False)

print("\n✓ All NLTK data downloaded!")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.



✓ All NLTK data downloaded!


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [13]:
# Create output directory
print("\n2. Creating output directory...")
import os
os.makedirs('output', exist_ok=True)
print("   ✓ Created")


2. Creating output directory...
   ✓ Created


In [14]:
!python main.py

Use Model: CodeLlama-7B-Instruct
2025-11-18 16:09:30.946914: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-18 16:09:30.963275: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763482170.984656    3890 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763482170.991134    3890 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763482171.007415    3890 computation_placer.cc:177] computation placer already registered. Please

In [18]:
import glob
import json

# Find the output file
output_files = glob.glob('output/*.json')
print(f"Output file: {output_files[0]}\n")

# Quick stats
with open(output_files[0], 'r') as f:
    results = json.load(f)

results = [r for r in results if r is not None]
print(f"Total examples processed: {len(results)}")
print(f"Avg time per example: {sum(r['time'] for r in results)/len(results):.2f}s")
print(f"Total time: {sum(r['time'] for r in results):.2f}s\n")

# Show first few questions
print("Questions processed:")
for i, r in enumerate(results[:5]):
    print(f"  {i+1}. {r['question'][:60]}...")

Output file: output/CodeLlama-7B-Instruct-spider-gtrr_column-S30C200-SS4CS6-L5-q_skltn-sql_skltn-sql_post_process-fs8-dev.json

Total examples processed: 779
Avg time per example: 18.78s
Total time: 14629.88s

Questions processed:
  1. How many singers do we have?...
  2. What is the total number of singers?...
  3. Show name, country, age for all singers ordered by age from ...
  4. What are the names, countries, and ages for every singer in ...
  5. What is the average, minimum, and maximum age of all singers...


In [21]:
%%writefile evaluate.py
#!/usr/bin/env python3
"""
Enhanced evaluation script for Gen-SQL with normalized EM and EX metrics
Properly counts null/failed examples as incorrect predictions
"""

import json
import sqlite3
import glob
import re


def normalize_sql(sql):
    """Normalize SQL for fair comparison"""
    if not sql:
        return ""

    sql = sql.lower().strip()
    sql = sql.rstrip(';').strip()
    sql = re.sub(r'\s+', ' ', sql)
    sql = re.sub(r'\s*,\s*', ', ', sql)
    sql = re.sub(r'\s*\(\s*', '(', sql)
    sql = re.sub(r'\s*\)\s*', ')', sql)

    return sql.strip()


def load_ground_truth(dev_file='benchmark/spider/dev.json'):
    """Load ground truth from Spider dev.json"""
    try:
        with open(dev_file, encoding='utf-8') as f:
            dev_data = json.load(f)

        # Create lookup by index (since results are ordered)
        ground_truth_by_index = {i: item for i, item in enumerate(dev_data)}

        # Also create lookup by (question, db_id) as backup
        ground_truth_by_key = {}
        for item in dev_data:
            key = (item['question'].strip(), item['db_id'])
            ground_truth_by_key[key] = item

        print(f"✓ Loaded {len(dev_data)} ground truth examples from {dev_file}")
        return ground_truth_by_index, ground_truth_by_key, dev_data
    except FileNotFoundError:
        print(f"❌ Ground truth file not found: {dev_file}")
        return {}, {}, []


def calculate_metrics(all_results, ground_truth_by_index, ground_truth_by_key):
    """Calculate EM and EX with proper handling of null/failed examples"""

    total_examples = len(all_results)
    valid_results = [r for r in all_results if r and isinstance(r, dict)]
    null_count = total_examples - len(valid_results)

    print("\n" + "="*70)
    print("EVALUATION SUMMARY")
    print("="*70)
    print(f"Total examples:          {total_examples}")
    print(f"Successful generations:  {len(valid_results)}")
    print(f"Failed (null):           {null_count}")

    # Track metrics
    em_matches = 0
    ex_correct = 0
    ex_errors = []

    examples_for_display = []

    for i, result in enumerate(all_results):
        # Get ground truth for this index
        if i not in ground_truth_by_index:
            continue

        gt_item = ground_truth_by_index[i]
        gold_sql = gt_item['query']
        db_id = gt_item['db_id']
        question = gt_item['question']

        # Handle null/failed results
        if not result or not isinstance(result, dict):
            # Null result = both EM and EX are wrong
            ex_errors.append({
                'example': i + 1,
                'question': question,
                'db': db_id,
                'error': 'Null result - generation failed'
            })
            continue

        # Get predicted SQL
        pred_sql = result.get('infer', '').strip()
        if not pred_sql:
            ex_errors.append({
                'example': i + 1,
                'question': question,
                'db': db_id,
                'error': 'Empty SQL prediction'
            })
            continue

        # Calculate EM
        norm_pred = normalize_sql(pred_sql)
        norm_gold = normalize_sql(gold_sql)
        is_em_match = (norm_pred == norm_gold)
        if is_em_match:
            em_matches += 1

        # Calculate EX
        db_path = f"benchmark/spider/database/{db_id}/{db_id}.sqlite"
        try:
            conn = sqlite3.connect(db_path)
            cursor = conn.cursor()

            cursor.execute(pred_sql)
            pred_results = cursor.fetchall()

            cursor.execute(gold_sql)
            gold_results = cursor.fetchall()

            is_ex_correct = (pred_results == gold_results)
            if is_ex_correct:
                ex_correct += 1
            else:
                ex_errors.append({
                    'example': i + 1,
                    'question': question,
                    'db': db_id,
                    'reason': 'Results mismatch',
                    'pred_rows': len(pred_results),
                    'gold_rows': len(gold_results)
                })

            conn.close()

        except sqlite3.Error as e:
            ex_errors.append({
                'example': i + 1,
                'question': question,
                'db': db_id,
                'error': f'SQL error: {str(e)[:100]}'
            })
        except FileNotFoundError:
            ex_errors.append({
                'example': i + 1,
                'question': question,
                'db': db_id,
                'error': 'Database file not found'
            })

        # Store first 5 for display
        if len(examples_for_display) < 5:
            examples_for_display.append({
                'num': i + 1,
                'question': question[:60] + '...',
                'gold': norm_gold[:70] + '...' if len(norm_gold) > 70 else norm_gold,
                'pred': norm_pred[:70] + '...' if len(norm_pred) > 70 else norm_pred,
                'em': '✓' if is_em_match else '✗',
                'ex': '✓' if is_ex_correct else '✗'
            })

    # Calculate accuracies
    em_accuracy = (em_matches / total_examples * 100) if total_examples > 0 else 0
    ex_accuracy = (ex_correct / total_examples * 100) if total_examples > 0 else 0

    # Also calculate conditional accuracies (on successful generations only)
    em_cond = (em_matches / len(valid_results) * 100) if len(valid_results) > 0 else 0
    ex_cond = (ex_correct / len(valid_results) * 100) if len(valid_results) > 0 else 0

    return {
        'total': total_examples,
        'valid': len(valid_results),
        'null': null_count,
        'em_matches': em_matches,
        'em_accuracy': em_accuracy,
        'em_conditional': em_cond,
        'ex_correct': ex_correct,
        'ex_accuracy': ex_accuracy,
        'ex_conditional': ex_cond,
        'ex_errors': ex_errors,
        'examples': examples_for_display
    }


def main():
    """Main evaluation pipeline"""
    print("="*70)
    print("GEN-SQL EVALUATION: Normalized EM + EX")
    print("(Null/Failed predictions count as incorrect)")
    print("="*70)

    # Find latest output
    pattern = 'output/*-spider-*.json'
    files = [f for f in glob.glob(pattern)
             if '_report' not in f and '_errors' not in f]

    if not files:
        print(f"❌ No results found matching: {pattern}")
        return

    results_file = files[0]
    print(f"\n📂 Results: {results_file}")

    # Load results
    with open(results_file, encoding='utf-8') as f:
        all_results = json.load(f)

    # Load ground truth
    gt_by_idx, gt_by_key, gt_list = load_ground_truth()
    if not gt_by_idx:
        print("❌ Cannot evaluate without ground truth!")
        return

    # Calculate metrics
    metrics = calculate_metrics(all_results, gt_by_idx, gt_by_key)

    # Display results
    print("\n" + "="*70)
    print("📊 EXACT MATCH (EM) - Normalized")
    print("="*70)
    print(f"Correct: {metrics['em_matches']}/{metrics['total']}")
    print(f"Overall Accuracy: {metrics['em_accuracy']:.1f}%")
    print(f"  (includes {metrics['null']} failures as incorrect)")
    print(f"\nConditional Accuracy: {metrics['em_conditional']:.1f}%")
    print(f"  (only on {metrics['valid']} successful generations)")

    print("\n" + "="*70)
    print("📊 EXECUTION ACCURACY (EX)")
    print("="*70)
    print(f"Correct: {metrics['ex_correct']}/{metrics['total']}")
    print(f"Overall Accuracy: {metrics['ex_accuracy']:.1f}%")
    print(f"  (includes {metrics['null']} failures as incorrect)")
    print(f"\nConditional Accuracy: {metrics['ex_conditional']:.1f}%")
    print(f"  (only on {metrics['valid']} successful generations)")
    print(f"\nExecution errors: {len(metrics['ex_errors'])}")

    # Show examples
    print("\n" + "="*70)
    print("📋 First 5 examples:")
    print("="*70)
    for ex in metrics['examples']:
        print(f"\n{ex['num']}. {ex['question']}")
        print(f"   Gold: {ex['gold']}")
        print(f"   Pred: {ex['pred']}")
        print(f"   EM: {ex['em']}, EX: {ex['ex']}")

    # Final summary
    print("\n" + "="*70)
    print("FINAL RESULTS")
    print("="*70)
    print(f"Dataset: Spider dev ({metrics['total']} examples)")
    print(f"Successful generations: {metrics['valid']} ({metrics['valid']/metrics['total']*100:.1f}%)")
    print(f"Failed generations: {metrics['null']} ({metrics['null']/metrics['total']*100:.1f}%)")
    print()
    print(f"Overall Exact Match (EM):        {metrics['em_accuracy']:.1f}%")
    print(f"Overall Execution Accuracy (EX): {metrics['ex_accuracy']:.1f}%")
    print()
    print(f"Conditional EM (on success):     {metrics['em_conditional']:.1f}%")
    print(f"Conditional EX (on success):     {metrics['ex_conditional']:.1f}%")
    print()
    if metrics['ex_accuracy'] > metrics['em_accuracy']:
        print(f"✓ EX better than EM: +{metrics['ex_accuracy'] - metrics['em_accuracy']:.1f}%")
    print("="*70)

    # Save errors
    if metrics['ex_errors']:
        error_file = results_file.replace('.json', '_errors.json')
        with open(error_file, 'w', encoding='utf-8') as f:
            json.dump(metrics['ex_errors'], f, indent=2)
        print(f"\n💾 Errors saved to: {error_file}")


if __name__ == '__main__':
    main()

Overwriting evaluate.py


In [22]:
!python evaluate.py

GEN-SQL EVALUATION: Normalized EM + EX
(Null/Failed predictions count as incorrect)

📂 Results: output/CodeLlama-7B-Instruct-spider-gtrr_column-S30C200-SS4CS6-L5-q_skltn-sql_skltn-sql_post_process-fs8-dev.json
✓ Loaded 1034 ground truth examples from benchmark/spider/dev.json

EVALUATION SUMMARY
Total examples:          1034
Successful generations:  779
Failed (null):           255

📊 EXACT MATCH (EM) - Normalized
Correct: 331/1034
Overall Accuracy: 32.0%
  (includes 255 failures as incorrect)

Conditional Accuracy: 42.5%
  (only on 779 successful generations)

📊 EXECUTION ACCURACY (EX)
Correct: 563/1034
Overall Accuracy: 54.4%
  (includes 255 failures as incorrect)

Conditional Accuracy: 72.3%
  (only on 779 successful generations)

Execution errors: 471

📋 First 5 examples:

1. How many singers do we have?...
   Gold: select count(*)from singer
   Pred: select count(*)from singer
   EM: ✓, EX: ✓

2. What is the total number of singers?...
   Gold: select count(*)from singer
   Pred: 